# L3.M11 — Vector Index Sharding for Multi-Tenant SaaS

## Learning Arc

**Purpose:** Master production-grade vector database sharding to scale beyond single-index limitations (100 namespaces, 1M+ vectors) using consistent hashing and Redis-backed routing.

**Concepts Covered:**
- Consistent hashing with MurmurHash3 for deterministic tenant routing
- Single-tenant vs cross-shard query patterns and their latency trade-offs
- Production monitoring thresholds (300K vectors/shard, 18/20 namespaces)
- Hot shard detection and blue-green rebalancing strategies
- Decision framework: when to shard vs when namespace isolation suffices

**After Completing:**
You'll understand when sharding is justified (>80 tenants, >1M vectors, P95 >500ms) and how to implement it without sacrificing query performance. You'll recognize that most SaaS applications never need sharding—namespace isolation from M11.1-M11.3 handles the majority of use cases.

**Context in Track L3.M11:**
This module extends M11.1 (Tenant Isolation) and M11.2 (Tenant Metadata) by adding horizontal scaling when namespace-based isolation reaches platform limits. Critical for understanding enterprise SaaS architecture trade-offs.

In [ ]:
# Setup
import sys
import json
from config import Config, get_clients
from src.l3_m11_vector_index_sharding import ShardManager, ShardedRAG, monitor_shard_health

# Initialize clients
pinecone_client, redis_client, openai_client = get_clients()
print(f"✓ Services: Pinecone={pinecone_client is not None}, Redis={redis_client is not None}, OpenAI={openai_client is not None}")
print(f"✓ Configuration: {Config.NUM_SHARDS} shards, {Config.VECTOR_DIMENSION}D vectors")

# Expected:
# ✓ Services: Pinecone=True/False, Redis=True/False, OpenAI=True/False
# ✓ Configuration: 4 shards, 1536D vectors

# Setup
import sys
import json
import os
from config import Config, get_clients
from src.l3_m11_vector_index_sharding import ShardManager, ShardedRAG, monitor_shard_health

# OFFLINE mode for L3 consistency
OFFLINE = os.getenv("OFFLINE", "false").lower() == "true"
if OFFLINE:
    print("⚠️  Running in OFFLINE mode — API calls to /query or /ingest will be skipped (mocked).")

# Initialize clients
pinecone_client, redis_client, openai_client = get_clients()
print(f"✓ Services: Pinecone={pinecone_client is not None}, Redis={redis_client is not None}, OpenAI={openai_client is not None}")
print(f"✓ Configuration: {Config.NUM_SHARDS} shards, {Config.VECTOR_DIMENSION}D vectors")

# Expected:
# ✓ Services: Pinecone=True/False, Redis=True/False, OpenAI=True/False
# ✓ Configuration: 4 shards, 1536D vectors

In [ ]:
# Demonstrate consistent hashing
shard_manager = ShardManager(redis_client, num_shards=Config.NUM_SHARDS)

tenants = ["tenant-001", "tenant-002", "tenant-003", "tenant-004", "tenant-005"]
for tenant_id in tenants:
    shard_id = shard_manager.get_shard_for_tenant(tenant_id)
    index_name = shard_manager.get_shard_index_name(tenant_id)
    print(f"{tenant_id} -> shard {shard_id} ({index_name})")

# Expected:
# tenant-001 -> shard X (tenant-shard-X)
# tenant-002 -> shard Y (tenant-shard-Y)
# ... deterministic routing to 0-3

## 2. Ingesting Documents to Shards

Each tenant's documents route to their assigned shard. Within the shard index, each tenant gets their own **namespace** (preserving isolation from M11.1-M11.3).

In [ ]:
## 2. Ingesting Documents to Shards

Each tenant's documents route to their assigned shard using the ShardManager's consistent hash. Within the shard index, each tenant gets their own **namespace** (preserving isolation from M11.1-M11.3).

**Architecture:** `tenant → hash → shard_id → index → namespace(tenant_id)`

This two-level isolation (shard + namespace) enables both horizontal scaling and tenant security.

## 3. Single-Tenant Query (Fast Path)

**Best practice:** Most queries should be single-tenant.

Hits only one shard, typical **P95 latency ~350ms**. This is the primary use case for production systems.

In [ ]:
## 3. Single-Tenant Query (Fast Path)

**Best practice:** 99% of production queries should be single-tenant.

Hits only one shard using the deterministic routing. Typical **P95 latency ~350ms**—comparable to non-sharded systems. The ShardManager routes the query to the correct index, then queries only that tenant's namespace.

## 4. Cross-Shard Query (Admin Use Only)

**Use sparingly:** Cross-shard queries are slower and should be limited to admin/analytics operations.

Aggregates results from all shards. Higher latency due to multiple index queries + result merging.

In [ ]:
# Cross-shard query (admin operation)
admin_query = data['queries'][2]  # No tenant_id - searches all shards
query_text = admin_query['query_text']

if pinecone_client and openai_client:
    result = rag.query_cross_shard(query_text, top_k=3)
    print(f"Cross-shard query: '{query_text}'")
    print(f"Shards queried: {result.get('shards_queried', 0)}, Latency: {result.get('latency_ms', 0):.0f}ms")
    print(f"Total results: {len(result.get('results', []))}")
else:
    print("⚠️ Skipping cross-shard query (no API keys)")

# Expected:
# Cross-shard query: 'Best practices for monitoring'
# Shards queried: 4, Latency: >800ms (slower than single-tenant)
# Total results: 0-6 matches (aggregated)

## 5. Monitoring Shard Health

**Production thresholds:**
- Vector count: Rebalance at **300K/shard**
- Namespace count: Alert at **18/20** capacity
- P95 latency: Warn at **500ms**

Monitor distribution to detect hot shards early.

In [ ]:
# Check shard distribution
stats = shard_manager.get_shard_stats()
print("Shard Distribution:")
for shard_id, info in stats.items():
    print(f"  Shard {shard_id}: {info['tenant_count']} tenants")

# Monitor health (requires Pinecone connection)
if pinecone_client:
    health = monitor_shard_health(shard_manager, pinecone_client)
    print(f"\nHealth: {'⚠️ Rebalancing needed' if health['needs_rebalancing'] else '✓ Healthy'}")
    if health['alerts']:
        for alert in health['alerts']:
            print(f"  Alert: {alert}")
else:
    print("\n⚠️ Skipping health check (no Pinecone)")

# Expected:
# Shard Distribution:
#   Shard 0: X tenants
#   Shard 1: Y tenants...

## 6. When This Breaks: Hot Shard Problem

**Symptom:** One shard accumulates large tenants, degrading performance.

**Cause:** 
- Uneven tenant growth over time
- Hash collision concentrates heavy users
- No rebalancing strategy in place

**Fix:** Explicit tenant reassignment for rebalancing.

In [ ]:
# Simulate hot shard detection and rebalancing
# Assume shard 0 is overloaded, move a tenant to underutilized shard 3

if redis_client:
    # Check current assignment
    tenant_to_move = "tenant-001"
    old_shard = shard_manager.get_shard_for_tenant(tenant_to_move)
    print(f"Current: {tenant_to_move} on shard {old_shard}")
    
    # Explicit reassignment (requires Redis)
    target_shard = 3
    shard_manager.assign_tenant_to_shard(tenant_to_move, target_shard)
    
    # Verify new assignment
    new_shard = shard_manager.get_shard_for_tenant(tenant_to_move)
    print(f"After rebalancing: {tenant_to_move} on shard {new_shard}")
else:
    print("⚠️ Rebalancing requires Redis")

# Expected:
# Current: tenant-001 on shard X
# After rebalancing: tenant-001 on shard 3

## 7. Decision Card: When to Shard

| Metric | Namespace Isolation | Sharded Architecture |
|--------|-------------------|---------------------|
| **Max Tenants** | ~100 | 1000s |
| **Query Latency (P95)** | 200-400ms | Single: 350ms, Cross: 800ms+ |
| **Operational Complexity** | Low | High |
| **Cross-Tenant Queries** | Fast | Slow (avoid) |
| **Rebalancing** | Not needed | Required skill |
| **Cost** | 1 index fee | N × index fees |

**Key Insight:** Start with namespace isolation (M11.1-M11.3). Only migrate to sharding when metrics prove necessity. Most SaaS apps never need sharding.

In [ ]:
# Decision helper function
def should_shard(num_tenants, total_vectors, p95_latency_ms, namespace_count):
    """Determine if sharding is justified."""
    reasons = []
    
    if num_tenants > 80:
        reasons.append(f"✓ {num_tenants} tenants (>80 threshold)")
    if total_vectors > 1_000_000:
        reasons.append(f"✓ {total_vectors:,} vectors (>1M threshold)")
    if namespace_count > 80:
        reasons.append(f"✓ {namespace_count} namespaces (approaching 100 limit)")
    if p95_latency_ms > 500:
        reasons.append(f"✓ {p95_latency_ms}ms P95 latency (>500ms threshold)")
    
    return len(reasons) > 0, reasons

# Test scenarios
scenarios = [
    ("Small SaaS", 30, 200_000, 250, 30),
    ("Growing SaaS", 85, 1_200_000, 550, 75),
]

for name, tenants, vectors, latency, namespaces in scenarios:
    should, reasons = should_shard(tenants, vectors, latency, namespaces)
    print(f"{name}: {'SHARD' if should else 'Use namespaces'}")
    for r in reasons:
        print(f"  {r}")

# Expected:
# Small SaaS: Use namespaces
# Growing SaaS: SHARD (with reasons listed)

## 8. Common Failures & Fixes

### Failure 1: Routing Inconsistency
**Symptom:** Same tenant routes to different shards across requests.

**Cause:** Redis connection lost, cache eviction, or inconsistent `num_shards` config.

**Fix:** Verify Redis connectivity and consistent configuration.

---

### Failure 2: Cross-Shard Query Timeout
**Symptom:** Admin queries timeout or return partial results.

**Cause:** High latency aggregating results from multiple indexes.

**Fix:** Lower `top_k` per shard, implement timeouts, or cache results.

---

### Failure 3: Rebalancing Disruption
**Symptom:** Tenant queries fail during rebalancing.

**Cause:** Moving data between shards without blue-green strategy.

**Fix:** Use blue-green rebalancing:
1. Write to both old and new shard
2. Backfill data to new shard
3. Switch reads to new shard
4. Clean up old shard

## Summary

**What we covered:**
1. ✓ Consistent hashing with MurmurHash3 for deterministic routing
2. ✓ Single-tenant queries (fast path ~350ms)
3. ✓ Cross-shard queries (slow, admin only)
4. ✓ Shard health monitoring with production thresholds
5. ✓ Hot shard problem detection and rebalancing
6. ✓ Decision framework for when to shard

**Key Trade-offs:**
- Operational complexity → Scalability to 1000s of tenants
- Single-tenant queries fast → Cross-shard queries slow
- Higher cost (N × indexes) → Overcome single-index limits

**Recommendation:** Start with namespace isolation. Migrate to sharding only when metrics justify the operational burden.

**Next Steps:**
- Configure `.env` with API keys (copy from `.env.example`)
- Run `PYTHONPATH=$PWD pytest -q` or `.\scripts\run_tests.ps1` to validate setup
- Start API with `PYTHONPATH=$PWD uvicorn app:app --reload` or `.\scripts\run_api.ps1`
- Monitor shard distribution and latency in production using `/metrics` endpoint